## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from keras.layers import BatchNormalization, Activation
from keras.utils import np_utils
from keras.optimizers import Adam
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/2D_spect/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/2D_spect/validate_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/2D_spect/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

data = trainD.reshape((trainD.shape[0], trainD.shape[1], trainD.shape[2], 1))
labels = np_utils.to_categorical(trainL, 4)

# data, labels = shuffle(trainData, trainLabel, random_state=0)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

training_data = []
training_label = []
validation_data = []
validation_label = []

for train_index, test_index in kf.split(data):
    print('trian:', train_index, 'len', len(train_index), 'test:', test_index, 'len', len(test_index))
    training_data.append(data[train_index])
    training_label.append(labels[train_index])
    validation_data.append(data[test_index])
    validation_label.append(labels[test_index])

training_data = np.array(training_data)
training_label = np.array(training_label)
validation_data = np.array(validation_data)
validation_label = np.array(validation_label)


KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    1     2     3 ... 10148 10149 10150] len 8120 test: [    0     5    10 ... 10133 10138 10146] len 2031
trian: [    0     2     3 ... 10148 10149 10150] len 8121 test: [    1    13    14 ... 10137 10142 10145] len 2030
trian: [    0     1     2 ... 10146 10148 10150] len 8121 test: [    6     7     9 ... 10136 10147 10149] len 2030
trian: [    0     1     4 ... 10147 10149 10150] len 8121 test: [    2     3     8 ... 10131 10144 10148] len 2030
trian: [    0     1     2 ... 10147 10148 10149] len 8121 test: [    4    24    26 ... 10141 10143 10150] len 2030


In [3]:
training_data[0].shape

(8120, 65, 139, 1)

In [4]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import itertools

def create_model():
    model = Sequential() 
    model.add(Conv2D(filters = 32, kernel_size = 3, data_format="channels_last", input_shape = (65, 139, 1)))
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 32, kernel_size = 3, activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Flatten())
    model.add(Dense(160, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = "relu"))
    model.add(Dense(4, activation = "softmax"))

    return model

def calculate_avg_F1(scores):
    tmp = [0,0,0,0]
    for i in scores:
        tmp += i
    tmp /= 5
    return tmp

def run_model(batch_S, learning_rate):
    
    Scores = []
    for X_train, Y_train, X_val, Y_val in zip(training_data, training_label, validation_data, validation_label):

        model = create_model()
        adam = Adam(lr = learning_rate)
        model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
        early_stop = EarlyStopping(patience=25)
        train_history_1 = model.fit(x = X_train, 
                                    y = Y_train,
                                    epochs=100,
                                    validation_data=(X_val, Y_val),
                                    callbacks=[early_stop],
                                    batch_size=batch_S, 
                                    verbose=1)
        
        evaluation = model.evaluate(x = X_val, y = Y_val)
        print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evaluation[0], evaluation[1]))

        validation_prediction = model.predict_classes(X_val, batch_size=batch_S)
        score = f1_score(Y_val.argmax(axis=1), validation_prediction, average=None)
        Scores.append(score)
        del model
    
    result = calculate_avg_F1(Scores)
    F1_result.append(result)
    del Scores
    

        
F1_result = []
hyperparameter_index = []
batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]

for bat, lr in itertools.product(batch_S, learning_rate):
    hyperparameter_index.append([bat, lr])
    run_model(bat, lr)

print('Finish training!')

Train on 8120 samples, validate on 2031 samples
Epoch 1/100
8120/8120 [==============================] - 10s 1ms/step - loss: 1.0866 - acc: 0.5888 - val_loss: 1.1570 - val_acc: 0.5731
Epoch 2/100
8120/8120 [==============================] - 6s 743us/step - loss: 1.0070 - acc: 0.5905 - val_loss: 1.1479 - val_acc: 0.5731
Epoch 3/100
8120/8120 [==============================] - 6s 749us/step - loss: 1.0043 - acc: 0.5898 - val_loss: 1.1661 - val_acc: 0.5731
Epoch 4/100
8120/8120 [==============================] - 6s 764us/step - loss: 1.0042 - acc: 0.5905 - val_loss: 1.1434 - val_acc: 0.5731
Epoch 5/100
8120/8120 [==============================] - 6s 731us/step - loss: 1.0030 - acc: 0.5903 - val_loss: 1.1308 - val_acc: 0.5731
Epoch 6/100
8120/8120 [==============================] - 6s 775us/step - loss: 1.0004 - acc: 0.5905 - val_loss: 1.1361 - val_acc: 0.5731
Epoch 7/100
8120/8120 [==============================] - 6s 784us/step - loss: 0.9938 - acc: 0.5904 - val_loss: 1.1423 - val_acc: 0

8120/8120 [==============================] - 6s 795us/step - loss: 0.8181 - acc: 0.6581 - val_loss: 0.9280 - val_acc: 0.6371
Epoch 61/100
8120/8120 [==============================] - 7s 801us/step - loss: 0.8203 - acc: 0.6552 - val_loss: 0.9189 - val_acc: 0.6376
Epoch 62/100
8120/8120 [==============================] - 6s 794us/step - loss: 0.8143 - acc: 0.6595 - val_loss: 0.9030 - val_acc: 0.6494
Epoch 63/100
8120/8120 [==============================] - 6s 790us/step - loss: 0.8100 - acc: 0.6599 - val_loss: 0.9858 - val_acc: 0.6465
Epoch 64/100
8120/8120 [==============================] - 6s 793us/step - loss: 0.8114 - acc: 0.6611 - val_loss: 0.8929 - val_acc: 0.6391
Epoch 65/100
8120/8120 [==============================] - 6s 795us/step - loss: 0.8071 - acc: 0.6616 - val_loss: 0.8878 - val_acc: 0.6608
Epoch 66/100
8120/8120 [==============================] - 6s 792us/step - loss: 0.8006 - acc: 0.6681 - val_loss: 0.9205 - val_acc: 0.6529
Epoch 67/100
8120/8120 [=======================

8121/8121 [==============================] - 6s 736us/step - loss: 0.7905 - acc: 0.6827 - val_loss: 0.8787 - val_acc: 0.7020
Epoch 78/100
8121/8121 [==============================] - 6s 719us/step - loss: 0.7943 - acc: 0.6786 - val_loss: 0.9052 - val_acc: 0.6872
Epoch 79/100
8121/8121 [==============================] - 6s 750us/step - loss: 0.7964 - acc: 0.6768 - val_loss: 0.9361 - val_acc: 0.6818
Epoch 80/100
8121/8121 [==============================] - 6s 736us/step - loss: 0.7959 - acc: 0.6779 - val_loss: 0.8840 - val_acc: 0.6980
Epoch 81/100
8121/8121 [==============================] - 6s 717us/step - loss: 0.7849 - acc: 0.6843 - val_loss: 0.8489 - val_acc: 0.7025
Epoch 82/100
8121/8121 [==============================] - 6s 741us/step - loss: 0.7738 - acc: 0.6915 - val_loss: 0.8819 - val_acc: 0.7030
Epoch 83/100
8121/8121 [==============================] - 6s 749us/step - loss: 0.7722 - acc: 0.6897 - val_loss: 0.8431 - val_acc: 0.7123
Epoch 84/100
8121/8121 [=======================

8121/8121 [==============================] - 6s 703us/step - loss: 0.7104 - acc: 0.7101 - val_loss: 0.8165 - val_acc: 0.7123
Epoch 95/100
8121/8121 [==============================] - 6s 704us/step - loss: 0.7098 - acc: 0.7112 - val_loss: 0.7954 - val_acc: 0.7133
Epoch 96/100
8121/8121 [==============================] - 6s 712us/step - loss: 0.7103 - acc: 0.7085 - val_loss: 0.8039 - val_acc: 0.7094
Epoch 97/100
8121/8121 [==============================] - 6s 698us/step - loss: 0.7049 - acc: 0.7110 - val_loss: 0.8043 - val_acc: 0.7158
Epoch 98/100
8121/8121 [==============================] - 6s 687us/step - loss: 0.6986 - acc: 0.7154 - val_loss: 0.7857 - val_acc: 0.7163
Epoch 99/100
8121/8121 [==============================] - 6s 716us/step - loss: 0.7008 - acc: 0.7156 - val_loss: 0.7676 - val_acc: 0.7143
Epoch 100/100
2030/2030 [==============================] - 0s 207us/step
Loss: 0.799, Accuracy: 0.710


/home/hsiehch/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

8121/8121 [==============================] - 5s 644us/step - loss: 0.9141 - acc: 0.6145 - val_loss: 0.9471 - val_acc: 0.6222
Epoch 24/100
8121/8121 [==============================] - 5s 649us/step - loss: 0.9063 - acc: 0.6222 - val_loss: 0.9087 - val_acc: 0.6389
Epoch 25/100
8121/8121 [==============================] - 5s 636us/step - loss: 0.8851 - acc: 0.6350 - val_loss: 0.9055 - val_acc: 0.6606
Epoch 26/100
8121/8121 [==============================] - 5s 649us/step - loss: 0.8886 - acc: 0.6343 - val_loss: 0.9080 - val_acc: 0.6438
Epoch 27/100
8121/8121 [==============================] - 5s 624us/step - loss: 0.8841 - acc: 0.6367 - val_loss: 0.9044 - val_acc: 0.6478
Epoch 28/100
8121/8121 [==============================] - 5s 606us/step - loss: 0.8535 - acc: 0.6535 - val_loss: 0.8503 - val_acc: 0.6675
Epoch 29/100
8121/8121 [==============================] - 5s 630us/step - loss: 0.8357 - acc: 0.6574 - val_loss: 0.8231 - val_acc: 0.6739
Epoch 30/100
8121/8121 [=======================

8121/8121 [==============================] - 5s 664us/step - loss: 0.7108 - acc: 0.7175 - val_loss: 0.7216 - val_acc: 0.7202
Epoch 41/100
8121/8121 [==============================] - 5s 612us/step - loss: 0.6839 - acc: 0.7270 - val_loss: 0.7043 - val_acc: 0.7310
Epoch 42/100
8121/8121 [==============================] - 5s 629us/step - loss: 0.6701 - acc: 0.7298 - val_loss: 0.7048 - val_acc: 0.7335
Epoch 43/100
8121/8121 [==============================] - 5s 639us/step - loss: 0.6753 - acc: 0.7261 - val_loss: 0.6924 - val_acc: 0.7276
Epoch 44/100
8121/8121 [==============================] - 5s 646us/step - loss: 0.6664 - acc: 0.7269 - val_loss: 0.7014 - val_acc: 0.7251
Epoch 45/100
8121/8121 [==============================] - 5s 650us/step - loss: 0.6590 - acc: 0.7341 - val_loss: 0.6975 - val_acc: 0.7286
Epoch 46/100
8121/8121 [==============================] - 5s 659us/step - loss: 0.6584 - acc: 0.7334 - val_loss: 0.6883 - val_acc: 0.7291
Epoch 47/100
8121/8121 [=======================

8121/8121 [==============================] - 5s 658us/step - loss: 0.5864 - acc: 0.7638 - val_loss: 0.6637 - val_acc: 0.7379
Epoch 58/100
8121/8121 [==============================] - 5s 630us/step - loss: 0.5834 - acc: 0.7681 - val_loss: 0.6704 - val_acc: 0.7409
Epoch 59/100
8121/8121 [==============================] - 5s 665us/step - loss: 0.5791 - acc: 0.7706 - val_loss: 0.6774 - val_acc: 0.7355
Epoch 60/100
8121/8121 [==============================] - 5s 642us/step - loss: 0.5695 - acc: 0.7726 - val_loss: 0.6741 - val_acc: 0.7374
Epoch 61/100
8121/8121 [==============================] - 5s 628us/step - loss: 0.5643 - acc: 0.7780 - val_loss: 0.6738 - val_acc: 0.7443
Epoch 62/100
8121/8121 [==============================] - 5s 639us/step - loss: 0.5579 - acc: 0.7806 - val_loss: 0.6623 - val_acc: 0.7498
Epoch 63/100
8121/8121 [==============================] - 5s 644us/step - loss: 0.5575 - acc: 0.7795 - val_loss: 0.6689 - val_acc: 0.7345
Epoch 64/100
8121/8121 [=======================

8121/8121 [==============================] - 6s 678us/step - loss: 0.9966 - acc: 0.5859 - val_loss: 0.9770 - val_acc: 0.5916
Epoch 37/100
8121/8121 [==============================] - 5s 654us/step - loss: 0.9966 - acc: 0.5859 - val_loss: 0.9749 - val_acc: 0.5916
Epoch 38/100
2030/2030 [==============================] - 1s 403us/step
Loss: 0.975, Accuracy: 0.592
Train on 8121 samples, validate on 2030 samples
Epoch 1/100
8121/8121 [==============================] - 25s 3ms/step - loss: 1.0471 - acc: 0.5491 - val_loss: 1.0261 - val_acc: 0.5995
Epoch 2/100
8121/8121 [==============================] - 5s 615us/step - loss: 1.0014 - acc: 0.5839 - val_loss: 0.9959 - val_acc: 0.5995
Epoch 3/100
8121/8121 [==============================] - 5s 634us/step - loss: 0.9990 - acc: 0.5839 - val_loss: 0.9997 - val_acc: 0.5995
Epoch 4/100
8121/8121 [==============================] - 5s 616us/step - loss: 0.9946 - acc: 0.5839 - val_loss: 0.9957 - val_acc: 0.5995
Epoch 5/100
8121/8121 [==================

8121/8121 [==============================] - 5s 675us/step - loss: 0.9917 - acc: 0.5869 - val_loss: 0.9977 - val_acc: 0.5877
Epoch 26/100
8121/8121 [==============================] - 5s 665us/step - loss: 0.9911 - acc: 0.5869 - val_loss: 0.9979 - val_acc: 0.5877
Epoch 27/100
8121/8121 [==============================] - 5s 647us/step - loss: 0.9912 - acc: 0.5869 - val_loss: 0.9983 - val_acc: 0.5877
Epoch 28/100
8121/8121 [==============================] - 5s 618us/step - loss: 0.9916 - acc: 0.5869 - val_loss: 0.9986 - val_acc: 0.5877
Epoch 29/100
8121/8121 [==============================] - 5s 619us/step - loss: 0.9918 - acc: 0.5869 - val_loss: 0.9983 - val_acc: 0.5877
Epoch 30/100
8121/8121 [==============================] - 5s 658us/step - loss: 0.9919 - acc: 0.5869 - val_loss: 1.0010 - val_acc: 0.5877
Epoch 31/100
8121/8121 [==============================] - 5s 636us/step - loss: 0.9921 - acc: 0.5869 - val_loss: 0.9988 - val_acc: 0.5877
Epoch 32/100
8121/8121 [=======================

In [7]:
np.set_printoptions(suppress=True)
for bat, lr in itertools.product(batch_S, learning_rate):
    print(bat, lr)

30 5e-05
30 0.0001
30 0.0005
30 0.001
30 0.005
50 5e-05
50 0.0001
50 0.0005
50 0.001
50 0.005
70 5e-05
70 0.0001
70 0.0005
70 0.001
70 0.005
90 5e-05
90 0.0001
90 0.0005
90 0.001
90 0.005
120 5e-05
120 0.0001
120 0.0005
120 0.001
120 0.005


In [6]:
import pprint
pprint.pprint(F1_result)

[array([0.06486289, 0.41684553, 0.83719154, 0.55882079]),
 array([0.24181083, 0.47240244, 0.86929866, 0.62601284]),
 array([0.53775161, 0.48532556, 0.87156565, 0.62260467]),
 array([0.16879158, 0.21155638, 0.79089031, 0.20674155]),
 array([0.        , 0.        , 0.73975146, 0.        ]),
 array([0.00199005, 0.39440208, 0.81634704, 0.5067441 ]),
 array([0.0965417 , 0.50026016, 0.84760723, 0.59630425]),
 array([0.64628924, 0.53980048, 0.87973937, 0.66307268]),
 array([0.        , 0.        , 0.73975146, 0.        ]),
 array([0.        , 0.        , 0.73975146, 0.        ]),
 array([0.02352941, 0.34846949, 0.7755278 , 0.27903612]),
 array([0.08153575, 0.43796272, 0.8363106 , 0.55894529]),
 array([0.51086356, 0.48359796, 0.86163063, 0.61233717]),
 array([0.23161248, 0.20033956, 0.79119413, 0.25128605]),
 array([0.        , 0.        , 0.73975146, 0.        ]),
 array([0.        , 0.37185782, 0.78065054, 0.26991078]),
 array([0.00855615, 0.46934648, 0.84537303, 0.56957039]),
 array([0.5460

### 補丁版

In [1]:
import numpy as np
from scipy.signal import spectrogram
import pylab as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from keras.layers import BatchNormalization, Activation
from keras.utils import np_utils
from keras.optimizers import Adam
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
np.set_printoptions(suppress=True)
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import itertools

def to_Spectrogram(data):
    _ , _ , Sx = spectrogram(data, fs=300, nperseg=300, noverlap=150, scaling='spectrum', detrend=False, window='hanning')
    Sx = np.log10(Sx)
    tmp = np.transpose(Sx)
    return tmp

trainD = np.load("/home/hsiehch/30s/train_data.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")

trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainD = np.append(trainD, testD, axis=0)

trainL = np.append(trainL, validationL, axis=0)
trainL = np.append(trainL, testL, axis=0)

data = []
labels = []

for i in range(len(trainD)):
    record = to_Spectrogram(trainD[i])
    data.append(record)
#     data.append(trainD[i])
    labels.append(trainL[i])

data = np.array(data)
labels = np.array(labels)

def foo(data, labels):
    
    type0 = []
    type1 = []
    type2 = []
    type3 = []
    
    L0 = []
    L1 = []
    L2 = []
    L3 = []
    
    for i in range(len(labels)):
        if labels[i] == 0:
            type0.append(data[i])
            L0.append(labels[i])
        if labels[i] == 1:
            type1.append(data[i])
            L1.append(labels[i])
        if labels[i] == 2:
            type2.append(data[i])
            L2.append(labels[i])
        if labels[i] == 3:
            type3.append(data[i])
            L3.append(labels[i])
        
    type0 = np.array(type0)
    type1 = np.array(type1)
    type2 = np.array(type2)
    type3 = np.array(type3)
    
    L0 = np.array(L0)
    L1 = np.array(L1)
    L2 = np.array(L2)
    L3 = np.array(L3)
    
    type0 = np.append(type0, type1, axis=0)
    type0 = np.append(type0, type2, axis=0)
    type0 = np.append(type0, type3, axis=0)
    
    L0 = np.append(L0, L1, axis=0)
    L0 = np.append(L0, L2, axis=0)
    L0 = np.append(L0, L3, axis=0)
    
    return type0, L0

trainD, trainL = foo(data, labels)

data = trainD.reshape((trainD.shape[0], trainD.shape[1], trainD.shape[2], 1))
labels = np_utils.to_categorical(trainL, 4)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

training_data = []
training_label = []
validation_data = []
validation_label = []

for train_index, test_index in kf.split(data):
    print('trian:', train_index, 'len', len(train_index), 'test:', test_index, 'len', len(test_index))
    training_data.append(data[train_index])
    training_label.append(labels[train_index])
    validation_data.append(data[test_index])
    validation_label.append(labels[test_index])

training_data = np.array(training_data)
training_label = np.array(training_label)
validation_data = np.array(validation_data)
validation_label = np.array(validation_label)


KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    1     2     3 ... 10146 10147 10149] len 8120 test: [    0     4     5 ... 10143 10148 10150] len 2031
trian: [    0     3     4 ... 10148 10149 10150] len 8121 test: [    1     2     6 ... 10132 10134 10142] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [   10    22    25 ... 10135 10141 10147] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [   15    16    19 ... 10140 10144 10145] len 2030
trian: [    0     1     2 ... 10147 10148 10150] len 8121 test: [    3     9    20 ... 10126 10146 10149] len 2030


In [3]:

def create_model():
    model = Sequential() 
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same', data_format="channels_last", input_shape = (59, 151, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Flatten())
    model.add(Dense(160, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = "relu"))
    model.add(Dense(4, activation = "softmax"))
    
#     print(model.summary())
    return model

def calculate_avg_F1(scores):
    tmp = [0,0,0,0]
    for i in scores:
        tmp += i
    tmp /= 5
    return tmp

def run_model(batch_S, learning_rate):
    
    Scores = []
    for X_train, Y_train, X_val, Y_val in zip(training_data, training_label, validation_data, validation_label):

        model = create_model()
        adam = Adam(lr = learning_rate)
        model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
        early_stop = EarlyStopping(patience=25)
        train_history_1 = model.fit(x = X_train, 
                                    y = Y_train,
                                    epochs=100,
                                    validation_data=(X_val, Y_val),
                                    callbacks=[early_stop],
                                    batch_size=batch_S, 
                                    verbose=1)
        
        evaluation = model.evaluate(x = X_val, y = Y_val)
        print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evaluation[0], evaluation[1]))

        validation_prediction = model.predict_classes(X_val, batch_size=batch_S)
        score = f1_score(Y_val.argmax(axis=1), validation_prediction, average=None)
        Scores.append(score)
        del model
    
    result = calculate_avg_F1(Scores)
    F1_result.append(result)
    del Scores
    

        
F1_result = []
hyperparameter_index = []
batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]

for bat, lr in itertools.product(batch_S, learning_rate):
    hyperparameter_index.append([bat, lr])
    run_model(bat, lr)

print('Finish training!')

Train on 8120 samples, validate on 2031 samples
Epoch 1/100
8120/8120 [==============================] - 14s 2ms/step - loss: 1.0765 - acc: 0.5491 - val_loss: 1.0620 - val_acc: 0.6036
Epoch 2/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.9632 - acc: 0.5869 - val_loss: 1.0765 - val_acc: 0.6578
Epoch 3/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.8748 - acc: 0.6312 - val_loss: 0.9575 - val_acc: 0.6716
Epoch 4/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.8112 - acc: 0.6681 - val_loss: 0.8478 - val_acc: 0.6686
Epoch 5/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.7752 - acc: 0.6810 - val_loss: 0.8332 - val_acc: 0.6869
Epoch 6/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.7551 - acc: 0.6882 - val_loss: 0.8572 - val_acc: 0.6425
Epoch 7/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.7349 - acc: 0.6966 - val_loss: 0.7895 - val_acc: 0.6775


8120/8120 [==============================] - 11s 1ms/step - loss: 0.3430 - acc: 0.8675 - val_loss: 0.5693 - val_acc: 0.7627
Epoch 61/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.3447 - acc: 0.8672 - val_loss: 0.5812 - val_acc: 0.7592
Epoch 62/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.3377 - acc: 0.8704 - val_loss: 0.5395 - val_acc: 0.7898
Epoch 63/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.3294 - acc: 0.8723 - val_loss: 0.5657 - val_acc: 0.7784
Epoch 64/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.3210 - acc: 0.8749 - val_loss: 0.5377 - val_acc: 0.8021
Epoch 65/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.3125 - acc: 0.8787 - val_loss: 0.5841 - val_acc: 0.7755
Epoch 66/100
8120/8120 [==============================] - 11s 1ms/step - loss: 0.3130 - acc: 0.8809 - val_loss: 0.5736 - val_acc: 0.7735
Epoch 67/100
8120/8120 [==============================

Epoch 20/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5735 - acc: 0.7749 - val_loss: 0.6568 - val_acc: 0.7586
Epoch 21/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5595 - acc: 0.7809 - val_loss: 0.6493 - val_acc: 0.7665
Epoch 22/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5565 - acc: 0.7796 - val_loss: 0.7075 - val_acc: 0.7483
Epoch 23/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5507 - acc: 0.7830 - val_loss: 0.6576 - val_acc: 0.7394
Epoch 24/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5416 - acc: 0.7871 - val_loss: 0.6634 - val_acc: 0.7606
Epoch 25/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5449 - acc: 0.7854 - val_loss: 0.7103 - val_acc: 0.7468
Epoch 26/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5318 - acc: 0.7905 - val_loss: 0.6299 - val_acc: 0.7729
Epoch 27/100
8121/8121 [=================

Epoch 80/100
2030/2030 [==============================] - 1s 448us/step
Loss: 0.523, Accuracy: 0.803
Train on 8121 samples, validate on 2030 samples
Epoch 1/100
8121/8121 [==============================] - 13s 2ms/step - loss: 1.1037 - acc: 0.5398 - val_loss: 1.1032 - val_acc: 0.5926
Epoch 2/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.9767 - acc: 0.5916 - val_loss: 0.9992 - val_acc: 0.6409
Epoch 3/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.8689 - acc: 0.6403 - val_loss: 0.9389 - val_acc: 0.6842
Epoch 4/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.8126 - acc: 0.6653 - val_loss: 0.8472 - val_acc: 0.6724
Epoch 5/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7691 - acc: 0.6837 - val_loss: 0.8200 - val_acc: 0.6749
Epoch 6/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7346 - acc: 0.7010 - val_loss: 0.7910 - val_acc: 0.6842
Epoch 7/100
8121/8121 [============

Epoch 59/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3561 - acc: 0.8617 - val_loss: 0.5480 - val_acc: 0.7936
Epoch 60/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3459 - acc: 0.8666 - val_loss: 0.5136 - val_acc: 0.8054
Epoch 61/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3440 - acc: 0.8668 - val_loss: 0.4977 - val_acc: 0.8133
Epoch 62/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3353 - acc: 0.8674 - val_loss: 0.5180 - val_acc: 0.8094
Epoch 63/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3310 - acc: 0.8717 - val_loss: 0.5163 - val_acc: 0.8123
Epoch 64/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3325 - acc: 0.8674 - val_loss: 0.5415 - val_acc: 0.8039
Epoch 65/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3157 - acc: 0.8802 - val_loss: 0.5379 - val_acc: 0.7921
Epoch 66/100
8121/8121 [=================

...

Epoch 95/100
2030/2030 [==============================] - 2s 746us/step
Loss: 0.792, Accuracy: 0.671
Train on 8121 samples, validate on 2030 samples
Epoch 1/100
8121/8121 [==============================] - 43s 5ms/step - loss: 1.1040 - acc: 0.5736 - val_loss: 1.1387 - val_acc: 0.5852
Epoch 2/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.9633 - acc: 0.5880 - val_loss: 1.0916 - val_acc: 0.5852
Epoch 3/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.9250 - acc: 0.6062 - val_loss: 0.9561 - val_acc: 0.6172
Epoch 4/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.8725 - acc: 0.6415 - val_loss: 1.1819 - val_acc: 0.4355
Epoch 5/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.8477 - acc: 0.6547 - val_loss: 0.9620 - val_acc: 0.6571
Epoch 6/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.8252 - acc: 0.6678 - val_loss: 0.9668 - val_acc: 0.6813
Epoch 7/100
8121/8121 [============

Epoch 59/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4696 - acc: 0.8223 - val_loss: 0.6652 - val_acc: 0.7552
Epoch 60/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4637 - acc: 0.8261 - val_loss: 0.6296 - val_acc: 0.7473
Epoch 61/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4626 - acc: 0.8281 - val_loss: 0.5798 - val_acc: 0.7842
Epoch 62/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4473 - acc: 0.8338 - val_loss: 0.6731 - val_acc: 0.7502
Epoch 63/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4478 - acc: 0.8304 - val_loss: 0.5665 - val_acc: 0.7951
Epoch 64/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4510 - acc: 0.8302 - val_loss: 0.8702 - val_acc: 0.6764
Epoch 65/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4554 - acc: 0.8240 - val_loss: 0.6156 - val_acc: 0.7581
Epoch 66/100
8121/8121 [=================

Epoch 24/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6596 - acc: 0.7350 - val_loss: 0.7376 - val_acc: 0.7015
Epoch 25/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6555 - acc: 0.7345 - val_loss: 0.7583 - val_acc: 0.7251
Epoch 26/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6469 - acc: 0.7309 - val_loss: 0.7190 - val_acc: 0.7187
Epoch 27/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6441 - acc: 0.7385 - val_loss: 0.7139 - val_acc: 0.7177
Epoch 28/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6481 - acc: 0.7365 - val_loss: 0.7399 - val_acc: 0.7163
Epoch 29/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6485 - acc: 0.7397 - val_loss: 1.4281 - val_acc: 0.3069
Epoch 30/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.6282 - acc: 0.7481 - val_loss: 0.7052 - val_acc: 0.7340
Epoch 31/100
8121/8121 [=================

Epoch 84/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4453 - acc: 0.8269 - val_loss: 0.5956 - val_acc: 0.7818
Epoch 85/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4569 - acc: 0.8232 - val_loss: 0.6733 - val_acc: 0.7360
Epoch 86/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4561 - acc: 0.8285 - val_loss: 0.6393 - val_acc: 0.7478
Epoch 87/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4444 - acc: 0.8283 - val_loss: 0.7918 - val_acc: 0.7217
Epoch 88/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4447 - acc: 0.8307 - val_loss: 0.8876 - val_acc: 0.6039
Epoch 89/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4416 - acc: 0.8344 - val_loss: 0.6077 - val_acc: 0.7606
Epoch 90/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.4390 - acc: 0.8322 - val_loss: 0.6820 - val_acc: 0.7562
Epoch 91/100
8121/8121 [=================

Epoch 43/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5428 - acc: 0.7923 - val_loss: 0.6754 - val_acc: 0.7300
Epoch 44/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5316 - acc: 0.7936 - val_loss: 0.6601 - val_acc: 0.7365
Epoch 45/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5331 - acc: 0.7918 - val_loss: 1.1697 - val_acc: 0.5483
Epoch 46/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5371 - acc: 0.7937 - val_loss: 0.6969 - val_acc: 0.7340
Epoch 47/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5132 - acc: 0.8003 - val_loss: 1.0813 - val_acc: 0.5276
Epoch 48/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5097 - acc: 0.8047 - val_loss: 1.4126 - val_acc: 0.4650
Epoch 49/100
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5126 - acc: 0.8009 - val_loss: 0.8612 - val_acc: 0.6488
Epoch 50/100
8121/8121 [=================

In [4]:
np.set_printoptions(suppress=True)
for bat, lr in itertools.product(batch_S, learning_rate):
    print(bat, lr)

30 5e-05
30 0.0001
30 0.0005
30 0.001
30 0.005
50 5e-05
50 0.0001
50 0.0005
50 0.001
50 0.005
70 5e-05
70 0.0001
70 0.0005
70 0.001
70 0.005
90 5e-05
90 0.0001
90 0.0005
90 0.001
90 0.005
120 5e-05
120 0.0001
120 0.0005
120 0.001
120 0.005


In [5]:
import pprint
pprint.pprint(F1_result)

[array([0.67951276, 0.5376521 , 0.87316773, 0.66677684]),
 array([0.6440876 , 0.51478626, 0.87033091, 0.66188905]),
 array([0.65552037, 0.52279823, 0.85821174, 0.66365676]),
 array([0.59333193, 0.49275888, 0.86766164, 0.64037382]),
 array([0.41852548, 0.26557852, 0.86119639, 0.57879868]),
 array([0.61746398, 0.55062997, 0.85564266, 0.65061513]),
 array([0.62853637, 0.53678958, 0.87161588, 0.66422555]),
 array([0.65771854, 0.47367678, 0.8589768 , 0.66640434]),
 array([0.61133593, 0.47211481, 0.86718088, 0.6461399 ]),
 array([0.56899408, 0.39985307, 0.86126336, 0.57020427]),
 array([0.61715535, 0.50782426, 0.85882982, 0.63957596]),
 array([0.58284772, 0.43792975, 0.87064606, 0.67242609]),
 array([0.63628775, 0.48830281, 0.86171517, 0.6323936 ]),
 array([0.56123733, 0.46431922, 0.85575347, 0.62617742]),
 array([0.15144117, 0.1293342 , 0.63019937, 0.25715518]),
 array([0.63156049, 0.48411279, 0.87806417, 0.65888121]),
 array([0.6172142 , 0.50342793, 0.85731134, 0.63963946]),
 array([0.6143